In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50, VGG16, VGG19, InceptionResNetV2
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [2]:

img_height = 150
img_width = 150
batch_size = 32
data_directory = '/Users/baudi/AI/practicas/uvas/data/train_val/'
test_data_directory = '/Users/baudi/AI/practicas/uvas/data/test/'
val_split = 0.2
seed = 42
num_classes = 4
learning_rate = 0.0001

In [3]:
# Función para ajustar las imágenes de entrenamiento
def adjustments(image):
    # Ajustar el brillo de la imagen de forma aleatoria entre -0.2 y 0.2
    image = tf.image.random_brightness(image, 0.2)
    # Ajustar el contraste de la imagen de forma aleatoria entre 0.9 y 1.1
    image = tf.image.random_contrast(image, 0.9, 1.1)
    return image

# Función para procesar las etiquetas
def process_labels(images, labels):
    # Convertir las etiquetas a one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_classes)
    return images, one_hot_labels

# Cargar los datos de entrenamiento
train_data_raw = image_dataset_from_directory(
    data_directory,                     # Directorio donde se encuentran las imágenes
    validation_split=val_split,         # Porcentaje del dataset a usar como validación
    subset="training",                  # Subconjunto de datos a utilizar
    seed=seed,                          # Semilla aleatoria para la reproducibilidad
    image_size=(img_height, img_width), # Tamaño de las imágenes
    batch_size=batch_size               # Tamaño del batch de entrenamiento
)

# Obtener los nombres de las clases
class_names = train_data_raw.class_names

# Aplicar las funciones de ajuste y procesamiento de etiquetas a los datos de entrenamiento
train_data = train_data_raw.map(lambda x, y: (adjustments(x), y)).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)

# Cargar los datos de validación
val_data = image_dataset_from_directory(
    data_directory,                     # Directorio donde se encuentran las imágenes
    validation_split=val_split,         # Porcentaje del dataset a usar como validación
    subset="validation",                # Subconjunto de datos a utilizar
    seed=seed,                          # Semilla aleatoria para la reproducibilidad
    image_size=(img_height, img_width), # Tamaño de las imágenes
    batch_size=batch_size               # Tamaño del batch de validación
).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)

# Cargar los datos de prueba
test_data = image_dataset_from_directory(
    test_data_directory,                # Directorio donde se encuentran las imágenes de prueba
    seed=seed,                          # Semilla aleatoria para la reproducibilidad
    image_size=(img_height, img_width), # Tamaño de las imágenes de prueba
    batch_size=batch_size               # Tamaño del batch de prueba
).map(process_labels).cache().prefetch(tf.data.experimental.AUTOTUNE)


Found 3248 files belonging to 4 classes.
Using 2599 files for training.
Metal device set to: Apple M2 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

Found 3248 files belonging to 4 classes.
Using 649 files for validation.
Found 814 files belonging to 4 classes.


2023-04-10 22:13:48.152207: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-04-10 22:13:48.152362: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
def build_model():
    # Cargar la red pre-entrenada VGG19
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
    
    # Congelar todas las capas de la red base excepto las últimas 12
    for layer in base_model.layers[:-12]:
        layer.trainable = False
        
    # Añadir capas adicionales para la clasificación
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)
    
    # Crear el modelo final
    model = Model(inputs=base_model.input, outputs=output)
    
    # Compilar el modelo
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [5]:
def build_model_custom():

    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(150, 150, 3)),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(256, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(512, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(1024, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(2048, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1024, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(4, activation='softmax')  # 4 clases: sanas, enfermedad1, enfermedad2, enfermedad3
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                loss='categorical_crossentropy',
                metrics=['accuracy'])
    
    return model

In [8]:
def cross_validate(model_builder, n_splits=2, epochs=20):
    # Crear un objeto KFold para dividir los datos en los folds de entrenamiento y validación
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    # Concatenar las imágenes y etiquetas de entrenamiento y validación en dos arrays separados
    X = np.concatenate([x for x, y in train_data] + [x for x, y in val_data])
    y = np.concatenate([y for x, y in train_data] + [y for x, y in val_data])

    # Lista para guardar las precisiones de validación de cada fold
    accuracies = []
    # Iterar sobre cada fold
    for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y), start=1):
        # Construir el modelo
        model = model_builder()
        print(f"Comienza el entrenamiento del fold {fold}")
        # Definir los callbacks de EarlyStopping y ReduceLROnPlateau
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)
        callbacks = [early_stopping, reduce_lr]

        # Entrenar el modelo en los datos del fold actual
        history = model.fit(X[train_idx],
                  y[train_idx],
                  validation_data=(X[val_idx], y[val_idx]),
                  batch_size = batch_size,
                  epochs=epochs,
                  callbacks=callbacks)
        print(f"Termina el entrenamiento del fold {fold}")
        # Guardar la precisión de validación del entrenamiento del modelo
        val_accuracy = history.history['val_accuracy']
        max_val_accuracy = max(val_accuracy)
        print(f"Precisión de validación del fold {fold}: {max_val_accuracy}")

        accuracies.append(max_val_accuracy)

    # Calcular y devolver la media de las precisiones de validación de todos los folds
    return np.mean(accuracies)

In [7]:
#Realizar la validación cruzada
mean_accuracy = cross_validate(build_model_custom,5,50)
print(f'Mean accuracy: {mean_accuracy:.4f}')


2023-04-10 22:13:48.405819: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Comienza el entrenamiento del fold 1
Epoch 1/50


2023-04-10 22:13:49.870494: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - ETA: 0s - loss: 1.0897 - accuracy: 0.6501

2023-04-10 22:13:55.623110: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


82/82 [==============================] - 7s 70ms/step - loss: 1.0897 - accuracy: 0.6501 - val_loss: 1.5333 - val_accuracy: 0.3585 - lr: 1.0000e-04
Epoch 2/50
82/82 [==============================] - 5s 62ms/step - loss: 0.6530 - accuracy: 0.7841 - val_loss: 0.7949 - val_accuracy: 0.6277 - lr: 1.0000e-04
Epoch 3/50
82/82 [==============================] - 5s 62ms/step - loss: 0.4650 - accuracy: 0.8414 - val_loss: 0.4839 - val_accuracy: 0.7985 - lr: 1.0000e-04
Epoch 4/50
82/82 [==============================] - 5s 62ms/step - loss: 0.3946 - accuracy: 0.8649 - val_loss: 0.2939 - val_accuracy: 0.8908 - lr: 1.0000e-04
Epoch 5/50
82/82 [==============================] - 5s 62ms/step - loss: 0.3309 - accuracy: 0.8853 - val_loss: 0.2275 - val_accuracy: 0.9138 - lr: 1.0000e-04
Epoch 6/50
82/82 [==============================] - 5s 61ms/step - loss: 0.2367 - accuracy: 0.9169 - val_loss: 0.2562 - val_accuracy: 0.8969 - lr: 1.0000e-04
Epoch 7/50
82/82 [==============================] - 5s 62ms/ste

NameError: name 'last_val_accuracy' is not defined

In [ ]:
def train_final_model(model_builder, epochs=20):
    # Crear el modelo final utilizando el modelo_builder
    final_model = model_builder()

    # Concatenar los datos de entrenamiento y validación
    X = np.concatenate([x for x, y in train_data])
    y = np.concatenate([y for x, y in train_data])

    # Definir el callback de EarlyStopping y ReduceLROnPlateau
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=0.00001)

    # Agregar los callbacks al método fit
    callbacks = [early_stopping, reduce_lr]
    final_model.fit(
        X,
        y, 
        epochs=epochs, 
        batch_size=batch_size, 
        validation_data=val_data, 
        callbacks=callbacks)

    # Devolver el modelo final entrenado
    return final_model

# Entrenar el modelo final utilizando build_model y 50 epochs
final_model = train_final_model(build_model_custom,50)


In [ ]:
# Evaluar el modelo en el conjunto de prueba
test_loss, test_accuracy = final_model.evaluate(test_data)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

# Predecir el conjunto de prueba
Y_pred = final_model.predict(test_data)
y_pred = np.argmax(Y_pred, axis=1)

# Obtener las etiquetas del conjunto de prueba
y_true = np.concatenate([y.numpy() for _, y in test_data.unbatch()])
y_true_labels = np.argmax(y_true.reshape(-1, len(class_names)), axis=1)

# Mostrar el informe de clasificación
print('Classification Report:')
print(classification_report(y_true_labels, y_pred, target_names=class_names))

# Mostrar la matriz de confusión
print('Confusion Matrix:')
print(confusion_matrix(y_true_labels, y_pred))
